In [ ]:
import sys
sys.path.append('..')  # this appends the parent directory to the sys.path list
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import tensorflow as tf
from nltk.util import ngrams
import itertools
from Utils import DataProcessing, FeatureCreation, Model
N_for_NGram = 3
Sequence_length = 200
epochs = 2
n_classes = 2

2023-03-12 03:23:43.479592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 03:23:43.613977: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 03:23:43.613999: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-12 03:23:44.194613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

ValueError: Please provide a TPU Name to connect to.

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")

In [ ]:
train_df, test_df = train_test_split(url_dataframe, test_size=0.20, stratify=url_dataframe['type'])
train_df, val_df = train_test_split(train_df, test_size=0.20, stratify=train_df['type'])

In [ ]:
# BATCH_SIZE = 16 * strategy.num_replicas_in_sync
Batch_Size = 64
max_tokens = 10000
steps_per_epoch = DataProcessing.steps_per_epoch(train_df, Batch_Size)
validation_steps = DataProcessing.steps_per_epoch(val_df, Batch_Size)
train_ds = DataProcessing.process_train_data(train_df,Batch_Size)
Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   max_tokens = max_tokens,
                                                   output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x,z: x[0])
Vectorize_Layer.adapt(train_text)
train_ds = train_ds.unbatch()
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.repeat()
train_ds = train_ds.batch(Batch_Size)
train_ds = train_ds.map(lambda x, z: DataProcessing.vectorize_text(x[0],x[1],z,Vectorize_Layer))
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_ds = DataProcessing.process_data(val_df,Batch_Size,Vectorize_Layer)
test_ds = DataProcessing.process_data(test_df,Batch_Size,Vectorize_Layer)

In [ ]:
max_tokens = len(Vectorize_Layer.get_vocabulary())

In [ ]:
# with strategy.scope():
model = Model.create_model(Sequence_length, max_tokens, 18)
print(model.summary())
early_stopping = tf.keras.callbacks.EarlyStopping(patience=40, monitor='val_loss', mode='min', restore_best_weights=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],steps_per_execution = 64,)

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=[early_stopping])

In [ ]:
from Utils import Evaluation
result = Evaluation.evaluate_model(model, test_ds, 0.5)